##  Sentinel-2 burned area identification

This notebook prepares the data transformation application: The outcome is a data transformation application that takes one input (or a set of inputs organized in an atomic unit) and generates the output.

The application implements:

* Calculation of NDVI in the two scenes (using band 8 and 4), (B8-B4)/(B8+B4)
* Calculation of NDWI, in the two scenes (using band 8 and 11), (B8-B11)/(B8+B11)
* If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

The outputs generated include:

* COG RGB composite with bands 12, 11, 8A pre
* COG RGB composite with bands 12, 11, 8A post 
* COG scene classification pre  
* COG scene classification post 
* COG 8 bits with bitmask burned/not burned
* Geojson with vectorization of bitmask burned/not burned

In [ ]:
service = dict([('title', 'Sentinel-2 burned area identification'),
                ('abstract', 'This is a short description'),
                ('id', 'ewf-satcen-03-03-02')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**


In [ ]:
pp_threshold = dict([('id', 'pp_threshold'),
                     ('title', 'Post Processing threshold in pixels'),
                     ('abstract', 'Number of pixels composing the isolated polygon to be removed (if 0 no post processing is applied)'),
                     ('value', '3'),
                     ('maxOccurs', '0'),
                     ('maxOccurs', '1')])

In [ ]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('title', 'NDVI difference threshold'),
                       ('abstract', 'NDVI difference threshold'),
                       ('maxOccurs', '1')]) 

In [ ]:
ndwi_threshold = dict([('id', 'ndwi_threshold'),
                   ('value', '0.18'),
                   ('title', 'NDWI difference threshold'),
                   ('abstract', 'NDWI difference threshold'),
                   ('maxOccurs', '1')])

In [ ]:
wkt = dict([('id', 'aoi'),
            ('value', 'POLYGON((149.74042460751588 -34.29772543048931,150.93246853304504 -34.323665099129535,150.90758708373184 -35.313155442237914,149.70124915286058 -35.28624837182783,149.74042460751588 -34.29772543048931))'),
            ('title', 'Area of interest'),
            ('abstract', 'Area of interest in WKT or bounding box')])

In [ ]:
input_references = ['https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191101T000241_N0213_R030_T56HKG_20191101T020007',
                    'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20200320T000241_N0214_R030_T56HKG_20200320T020042']

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

### Workflow

#### Import the packages

In [ ]:
import os
import sys
import shutil

import snappy
from snappy import ProductIO
from snappy import GPF
from snappy import WKTReader
S2CacheUtils = snappy.jpy.get_type('org.esa.s2tbx.dataio.cache.S2CacheUtils')
S2CacheUtils.deleteCache()

import cioppy
ciop = cioppy.Cioppy()

from datetime import datetime

import gdal
import geopandas as gp
import numpy as np
import datetime

sys.path.append(os.getcwd())
sys.path.append('/application/notebook/libexec/') 
from helpers import *

import warnings
warnings.filterwarnings('ignore')

gdal.UseExceptions()

In [ ]:
products = get_metadata(input_references, data_path)

In [ ]:
group_analysis(products)

In [ ]:
products

In [ ]:
req_bands = ['B4','B8','B8A', 'B11','B12', 'quality_scene_classification' ]

In [ ]:
geom = WKTReader().read(wkt['value'])

##### If more than one post or pre products==> use slice assembly ==>use new mosaics as input to the next steps

In [ ]:
output_files = []
for index, item in enumerate(['Pst','Pre']):
    if(products[products['ordinal_type'] == item].identifier.count()>1):
        product = mosaic_inputs(products[products['ordinal_type'] == item].reset_index(drop=True))
    else:
        local_pathx=products[products['ordinal_type'] == item].iloc[0]['local_path']
        s2prd = "%s/MTD_MSIL2A.xml" %local_pathx 
        product = snappy.ProductIO.readProduct(s2prd)
    
    output_name = '%s_%s.tif'%(item,product.getName())
    
    product = resample2ref_band(product,'B4')
    
    product = subset_to_aoi_reduce_bands(product,geom,req_bands)
   
    ProductIO.writeProduct(product, 'S2_{}_tmp.tif'.format(item), 'GeoTIFF-BigTIFF')
    
    output_files.append('RGB_{}'.format(output_name))
    snap_rgb(product,['B12','B11','B8A'],'RGB_{}'.format(output_name))
    
    product.dispose()
    S2CacheUtils.deleteCache()
    
print(output_files)

### NDVI & NDWI computation 

In [ ]:
ds = gdal.Open('S2_Pre_tmp.tif')
    
pre_b04 = ds.GetRasterBand(1).ReadAsArray()
pre_b08 = ds.GetRasterBand(2).ReadAsArray()
pre_b11 = ds.GetRasterBand(4).ReadAsArray()
pre_scl = ds.GetRasterBand(6).ReadAsArray()

ds = None

os.remove('S2_Pre_tmp.tif')

In [ ]:
ds = gdal.Open('S2_Pst_tmp.tif')
    
post_b04 = ds.GetRasterBand(1).ReadAsArray()
post_b08 = ds.GetRasterBand(2).ReadAsArray()
post_b11 = ds.GetRasterBand(4).ReadAsArray()
post_scl = ds.GetRasterBand(6).ReadAsArray()

width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()
#print(input_georef)
proj = osr.SpatialReference(wkt=ds.GetProjection())
espg = proj.GetAttrValue('AUTHORITY',1)
print(espg) 
ds = None

os.remove('S2_Pst_tmp.tif')

In [ ]:
ndvwi = lambda x,y: 0 if (x+y)==0  else float(x-y)/float(x+y)

vfunc = np.vectorize(ndvwi, otypes=[np.float32])

 ### NDWI with NIR (8) and SWIR (11)

In [ ]:
pre_ndwi2 = vfunc(pre_b08,pre_b11)
post_ndwi2 = vfunc(post_b08,post_b11)

pre_b11 = None
post_b11 = None

### NDVI with NIR (8) and Red (4)

In [ ]:
pre_ndvi = vfunc(pre_b08,pre_b04)
post_ndvi = vfunc(post_b08,post_b04)

pre_b04 = None
post_b04 = None

pre_b08 = None
post_b08 = None

#### Burned Area computation: 
#### If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

In [ ]:
ndwi_diff = pre_ndwi2  - post_ndwi2

In [ ]:
ndvi_diff = pre_ndvi - post_ndvi

In [ ]:
conditions = lambda x,y,z,m,n,p: 1 if ((x  > float(y)) & (z > float(m)) & ((n == 4) | (p == 4))) else 0
                             
vfunc_conditions = np.vectorize(conditions, otypes=[np.uint8])

In [ ]:
burned_0 = vfunc_conditions(ndwi_diff, ndwi_threshold['value'], ndvi_diff, ndvi_threshold['value'], pre_scl, post_scl )

In [ ]:
pre_ndwi2 = None
post_ndwi2 = None

pre_ndvi = None
post_ndvi = None

### Exclude according to scene classifications:

where noData put burned=2 if burn then put burned=1 else burned=0

In [ ]:
brnd = lambda x,y,z: 2 if (x==0 or y==0 or x==1 or y==1 or x==6 or y==6 or x==7 or y==7 or x==8 or y==8 or x==9 or y==9) else z

vfunc = np.vectorize(brnd, otypes=[np.uint8])

burned = vfunc(pre_scl , post_scl, burned_0 )

In [ ]:
burned_0 = None

##### Write the burned area temp tiff

In [ ]:
start_date=(products[products['ordinal_type'] == 'Pre'].reset_index(drop=True)).iloc[0].startdate#[0:10]

In [ ]:
end_date=(products[products['ordinal_type'] == 'Pst'].reset_index(drop=True)).iloc[0].enddate#[0:10]

In [ ]:
products.sort_values(by='startdate',ascending=True,inplace=True)

masterID = products.iloc[0].identifier
slaveID = products.iloc[1].identifier

In [ ]:
#Requested file name : 'Burned_Area_S2_{MasterId}_{SlaveId}.tif
if products[products['ordinal_type'] == 'Pre'].identifier.count() == 1 and products[products['ordinal_type'] == 'Pst'].identifier.count() == 1:
    temp_output_name_Burned_Area = 'temp_Burned_Area_S2_%s_%s.tif'%(masterID,slaveID)
else:
    #if inputs are mosaiced 
    temp_output_name_Burned_Area = 'temp_Burned_Area_S2_%s_%s.tif'%(start_date,end_date)

In [ ]:
write_tif(burned, temp_output_name_Burned_Area, width, height, input_geotransform, input_georef)

##### Post-processing step: removing raster polygons smaller than the provided threshold size (in pixels) - if threshold=0 no post-proc will be applied

In [ ]:
if int(pp_threshold['value']) != 0:
    
    output_name_Burned_Area = '_'.join(temp_output_name_Burned_Area.split('_')[1:])
    sieve_filter(temp_output_name_Burned_Area,
                 output_name_Burned_Area, 
                 int(pp_threshold['value']))
    os.remove(temp_output_name_Burned_Area)

else:
    shutil.move(temp_output_name_Burned_Area,
                output_name_Burned_Area)

In [ ]:
output_files.append(output_name_Burned_Area)

##### Creating the mask for the burned area to polygonize only Burned area polygons

In [ ]:
ds = gdal.Open(output_name_Burned_Area)
    
ba = ds.GetRasterBand(1).ReadAsArray()
ds=None

brnd_mask = lambda x: 1 if (x==1) else 0

vfunc = np.vectorize(brnd_mask, otypes=[np.uint8])

mask_burned_area = vfunc(ba)

write_tif(mask_burned_area, 'MASK_burned_area.tif', width, height, input_geotransform, input_georef)


In [ ]:
change_detection_gp = polygonize(output_name_Burned_Area, 1, espg, mask='MASK_burned_area.tif')

In [ ]:
output_files.append('polygonized.json')

#### if we replace {'init':'epsg:{}'.format(epsg)} with new recommended 'epsg:{}', the axis order changes

In [ ]:
change_detection_gp.head(10)

### Get the result WKT

In [ ]:
src = gdal.Open(output_name_Burned_Area)
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

min_x, min_y, max_x, max_y

In [ ]:
source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[1],
                 transform.TransformPoint(min_x, min_y)[0],
                 transform.TransformPoint(max_x, max_y)[1],
                 transform.TransformPoint(max_x, max_y)[0]).wkt

In [ ]:
result_wkt

### Create the properties file

In [ ]:
from datetime import datetime

In [ ]:
date_format = '%Y-%m-%dT%H:%m:%SZ'

In [ ]:
output_files

In [ ]:
for index , item in enumerate(output_files):

    if 'RGB' in item:
        prod = slaveID
        if 'Pre' in item[4:7]:
            prod = masterID
        title = 'Sentinel-2 RGB {}-event {} (B11, B12, B8A)'.format(item[4:7],prod)
            
    if 'Burned_Area_S2' in item:
        title = 'Sentinel-2 burned area identification for pair {}/{}'.format(masterID,slaveID)
        if 'temp_' in item:
            title = 'Sentinel-2 burned area identification for pair {}/{} (pre-filtering)'.format(masterID,slaveID)
    
    if 'polygonized' in item:
        title = 'Geojson with vectorization of bitmask burned=1/not burned=0/unkown=2 for pair {}/{}'.format(masterID,slaveID)
        
    
    with open('{}.properties'.format(item), 'w') as file:
        
        file.write('title={}\n'.format(title))
        
        if 'Pre-event' in title:
            start_date_iso = pd.to_datetime(products.iloc[0].startdate).strftime(date_format)
            end_date_iso = pd.to_datetime(products.iloc[0].enddate).strftime(date_format)
            file.write('date={}/{}\n'.format(start_date_iso,start_date_iso))
        elif 'Pst-event' in title :
            start_date_iso = pd.to_datetime(products.iloc[1].startdate).strftime(date_format)
            end_date_iso = pd.to_datetime(products.iloc[1].enddate).strftime(date_format)
            file.write('date={}/{}\n'.format(end_date_iso,end_date_iso))
        else:
            start_date_iso = pd.to_datetime(products.iloc[0].startdate).strftime(date_format)
            end_date_iso = pd.to_datetime(products.iloc[1].enddate).strftime(date_format)
            file.write('date={}/{}\n'.format(start_date_iso,end_date_iso))
            
        file.write('geometry={}'.format(result_wkt))


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.